In [34]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [35]:
!ls

drive  sample_data


In [41]:
# -*- coding: utf-8 -*-
# from __future__ import absolute_import
# from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from PIL import Image
import csv
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPooling2D, concatenate
from keras.optimizers import RMSprop, adam
from keras import backend as K
from sklearn.model_selection import train_test_split

epochs = 30
batch_size = 128

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)


def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)


def hinge_loss(y_true, y_pred):
    margin = K.constant(0.2)
    for i in range(0, batch_size, 2):
        s_pos = K.sum(y_pred[i])
        s_neg = K.sum(y_pred[i + 1])
    return K.maximum(K.constant(0), margin + s_neg - s_pos)


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    # print([len(digit_indices[d]) for d in range(num_classes)])
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_image_pairs(rows, ref, pos, neg):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    for i in range(rows):
        ref_patch, pos_patch = ref[i], pos[i]
        pairs += [[ref_patch, pos_patch]]
        ref_patch, neg_patch = ref[i], neg[i]
        pairs += [[ref_patch, neg_patch]]
        labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).'''
  

#     model = Sequential()
#     model.add(Conv2D(112, (3, 3), activation='relu', input_shape=input_shape))
#     model.add(Conv2D(112, (3, 3), activation='relu', padding='same'))
#     model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
#     model.add(Dropout(0.25))
#     model.add(Conv2D(112, (3, 3), activation='relu', padding='same'))
#     model.add(Conv2D(112, (3, 3), activation='relu', padding='same'))
#     model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
#     model.add(Dropout(0.25))
#     model.add(Conv2D(112, (3, 3), activation='relu', padding='same'))
#     model.add(Flatten())
#     model.add(Dropout(0.25))
#     model.add(Dense(384, activation='relu'))
   
    
#     print(model.summary())
#     return model  # Model(input, x)
  
  
  
  
  
    input_img = Input(shape = (11, 11, 1))
    tower_1 = Conv2D(64, (1,1), padding='same', activation='elu')(input_img)
    tower_1 = Conv2D(64, (3,3), padding='same', activation='elu')(tower_1)
    tower_2 = Conv2D(64, (1,1), padding='same', activation='elu')(input_img)
    tower_2 = Conv2D(64, (5,5), padding='same', activation='elu')(tower_2)
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)
    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    output = Flatten()(output)
#     out    = Dense(10, activation='softmax')(output)
    out    = Dense(128, activation='softmax')(output)
    
   

    
    
    
    
    model = Model(inputs = input_img, outputs = out)
    print(model.summary())
    return model


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() > 0.5
    return np.mean(pred == y_true)



def accuracy(y_true, y_pred):
    #Compute classification accuracy with a fixed threshold on distances.

    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


img_rows, img_cols = 11, 11

'''
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
'''
# big_file = '/content/drive/My Drive/yan/middlebury0-22.csv'
big_file = '/content/drive/My Drive/yan/446-451(5).csv'

csv_train = []
csv_test = []
csv_big = []

with open(big_file) as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        csv_big.append(row)
len_threshold = round(len(csv_big) * 0.80)
for i in range(len_threshold):
    csv_train.append(csv_big[i])
for i in range(len_threshold, len(csv_big)):
    csv_test.append(csv_big[i])
csv_big = np.array(csv_big)
csv_train = np.array(csv_train)
csv_test = np.array(csv_test)
[rows_tr, cols] = csv_train.shape
[rows_te, cols] = csv_test.shape
print('csv_big.shape', csv_big.shape)
print('csv_train.shape', csv_train.shape)
print('csv_test.shape', csv_test.shape)

ref_patches_train = csv_train[:, 0:121]
pos_patches_train = csv_train[:, 121:242]
neg_patches_train = csv_train[:, 242:]

ref_patches_new_train = np.zeros(shape=(rows_tr, 11, 11))
pos_patches_new_train = np.zeros(shape=(rows_tr, 11, 11))
neg_patches_new_train = np.zeros(shape=(rows_tr, 11, 11))

for i in range(rows_tr):
    ref_patches_new_train[i] = ref_patches_train[i].reshape((11, 11))
    pos_patches_new_train[i] = pos_patches_train[i].reshape((11, 11))
    neg_patches_new_train[i] = neg_patches_train[i].reshape((11, 11))

print('ref_patch_trains', ref_patches_new_train.shape)
'''
for i in range(10):
    img_ref = Image.fromarray(ref_patches_new_train[i])
    img_pos = Image.fromarray(pos_patches_new_train[i])
    img_neg = Image.fromarray(neg_patches_new_train[i])
    if img_ref.mode != 'RGB':
        img_ref = img_ref.convert('RGB')
    if  img_pos.mode != 'RGB':
        img_pos = img_pos.convert('RGB')
    if img_neg.mode != 'RGB':
        img_neg = img_neg.convert('RGB')
    name_ref = 'ref'+str(i)+'.png'
    name_pos = 'pos'+str(i)+'.png'
    name_neg = 'neg'+str(i)+'.png'
    img_ref.save(name_ref)
    img_pos.save(name_pos)
    img_neg.save(name_neg)
'''
ref_patches_test = csv_test[:, 0:121]
pos_patches_test = csv_test[:, 121:242]
neg_patches_test = csv_test[:, 242:]

ref_patches_new_test = np.zeros(shape=(rows_te, 11, 11))
pos_patches_new_test = np.zeros(shape=(rows_te, 11, 11))
neg_patches_new_test = np.zeros(shape=(rows_te, 11, 11))

print('ref_patch_test', ref_patches_new_test.shape)

for i in range(rows_te):
    ref_patches_new_test[i] = ref_patches_test[i].reshape((11, 11))
    pos_patches_new_test[i] = pos_patches_test[i].reshape((11, 11))
    neg_patches_new_test[i] = neg_patches_test[i].reshape((11, 11))
'''    
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

'''
# remove mean
# data augmentation
input_shape = (img_rows, img_cols, 1)
ref_patches_new_train = ref_patches_new_train.astype('float32')
pos_patches_new_train = pos_patches_new_train.astype('float32')
neg_patches_new_train = neg_patches_new_train.astype('float32')
ref_patches_new_train /= 255
pos_patches_new_train /= 255
neg_patches_new_train /= 255

ref_patches_new_test = ref_patches_new_test.astype('float32')
pos_patches_new_test = pos_patches_new_test.astype('float32')
neg_patches_new_test = neg_patches_new_test.astype('float32')
ref_patches_new_test /= 255
pos_patches_new_test /= 255
neg_patches_new_test /= 255

tr_pairs, tr_y = create_image_pairs(rows_tr, ref_patches_new_train, pos_patches_new_train, neg_patches_new_train)
tr_pairs = np.expand_dims(tr_pairs, axis=-1)
x_train, x_valid, y_train, y_valid = train_test_split(tr_pairs, tr_y, test_size = 0.3, shuffle= False)

print('tr_pairs shape:', tr_pairs[:, 0].shape)
print('tr_y shape:', tr_y.shape)
print(tr_y)

te_pairs, te_y = create_image_pairs(rows_te, ref_patches_new_test, pos_patches_new_test, neg_patches_new_test)
te_pairs = np.expand_dims(te_pairs, axis=-1)
print('te_pairs shape:', te_pairs[:, 1].shape)
print('te_y shape:', te_y.shape)
print(te_y)

# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# distance = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([processed_a, processed_b])
concatenate_vector = concatenate([processed_a, processed_b], axis=1)
# print('concatenate shape', tf.shape(concatenate_vector))

dense1 = Dense(384, name='dense1', activation='relu')(concatenate_vector)
dense2 = Dense(384, name='dense2', activation='relu')(dense1)
output = Dense(1, name='output_layer', activation='sigmoid')(dense2)

# distance = Lambda(euclidean_distance,
# output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model(inputs=[input_a, input_b], outputs=output)
print(model.summary())

# train
rms = RMSprop()
# rms = RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
# sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])

print(tr_pairs[:, 0].shape)
hist =    model.fit([x_train[:, 0], x_train[:, 1]], y_train,
          batch_size=128,
          epochs=epochs,
          validation_data=([x_valid[:, 0], x_valid[:, 1]], y_valid))

# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))






csv_big.shape (26629, 363)
csv_train.shape (21303, 363)
csv_test.shape (5326, 363)
ref_patch_trains (21303, 11, 11)
ref_patch_test (5326, 11, 11)
tr_pairs shape: (42606, 11, 11, 1)
tr_y shape: (42606,)
[1 0 1 ... 0 1 0]
te_pairs shape: (10652, 11, 11, 1)
te_y shape: (10652,)
[1 0 1 ... 0 1 0]
Model: "model_49"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_74 (InputLayer)           (None, 11, 11, 1)    0                                            
__________________________________________________________________________________________________
conv2d_126 (Conv2D)             (None, 11, 11, 64)   128         input_74[0][0]                   
__________________________________________________________________________________________________
conv2d_128 (Conv2D)             (None, 11, 11, 64)   128         input_74[0][0]               